In [3]:
pip install python-binance

     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 43 kB 1.0 MB/s 
     |████████████████████████████████| 103 kB 39.1 MB/s 
     |████████████████████████████████| 1.1 MB 9.0 MB/s 
     |████████████████████████████████| 288 kB 10.0 MB/s 
     |████████████████████████████████| 160 kB 56.0 MB/s 
     |████████████████████████████████| 192 kB 63.4 MB/s 
     |████████████████████████████████| 271 kB 29.2 MB/s 


In [1]:
pip install ta

  Created wheel for ta: filename=ta-0.8.0-py3-none-any.whl size=28895 sha256=9e48743298630df9389465ad7f561c6584e444f997951df105e495931592bab3
  Stored in directory: /root/.cache/pip/wheels/7e/da/86/65cba22446ae2ef148de2079907264ef27feecfb7f51a45e0d
Successfully built ta


In [4]:
import pandas as pd
from binance.client import Client
import ta



In [5]:
klinesT = Client().get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1HOUR, "01 January 2017")

In [7]:
df = pd.DataFrame(klinesT, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
print(df)

           timestamp            open  ...        tb_quote_av         ignore
0      1502942400000   4261.48000000  ...    150952.47794304  7887.63551305
1      1502946000000   4308.83000000  ...     92608.27972836  8039.26240152
2      1502949600000   4330.29000000  ...     20795.31722405  8041.76049845
3      1502953200000   4316.62000000  ...     11291.34701534  8048.12788573
4      1502956800000   4333.32000000  ...      3552.74681715  8018.03650010
...              ...             ...  ...                ...            ...
37964  1640066400000  48600.50000000  ...  33893956.58406380              0
37965  1640070000000  48546.88000000  ...  30650430.19022910              0
37966  1640073600000  48579.77000000  ...  72221708.86329550              0
37967  1640077200000  48566.43000000  ...  30000512.64933950              0
37968  1640080800000  48591.71000000  ...    191723.33890100              0

[37969 rows x 12 columns]


In [8]:
del df['ignore']
del df['close_time']
del df['quote_av']
del df['trades']
del df['tb_base_av']
del df['tb_quote_av']

df['close'] = pd.to_numeric(df['close'])
df['high'] = pd.to_numeric(df['high'])
df['low'] = pd.to_numeric(df['low'])
df['open'] = pd.to_numeric(df['open'])
df = df.set_index(df['timestamp'])
df.index = pd.to_datetime(df.index, unit='ms')

del df['timestamp']
print(df)

                         open      high       low     close         volume
timestamp                                                                 
2017-08-17 04:00:00   4261.48   4313.62   4261.32   4308.83    47.18100900
2017-08-17 05:00:00   4308.83   4328.69   4291.37   4315.32    23.23491600
2017-08-17 06:00:00   4330.29   4345.45   4309.37   4324.35     7.22969100
2017-08-17 07:00:00   4316.62   4349.99   4287.41   4349.99     4.44324900
2017-08-17 08:00:00   4333.32   4377.85   4333.32   4360.69     0.97280700
...                       ...       ...       ...       ...            ...
2021-12-21 06:00:00  48600.50  48704.17  48331.56  48546.88  1481.32305000
2021-12-21 07:00:00  48546.88  48671.56  48412.43  48579.77  1225.43448000
2021-12-21 08:00:00  48579.77  49000.00  48512.00  48566.44  2877.38845000
2021-12-21 09:00:00  48566.43  48699.00  48438.49  48591.72  1258.33927000
2021-12-21 10:00:00  48591.71  48625.00  48571.43  48579.75     9.43433000

[37969 rows x 5 columns]

In [9]:
df['SMA200'] = ta.trend.sma_indicator(df['close'], 200)
df['SMA600'] = ta.trend.sma_indicator(df['close'], 600)
print(df)

                         open      high  ...       SMA200        SMA600
timestamp                                ...                           
2017-08-17 04:00:00   4261.48   4313.62  ...          NaN           NaN
2017-08-17 05:00:00   4308.83   4328.69  ...          NaN           NaN
2017-08-17 06:00:00   4330.29   4345.45  ...          NaN           NaN
2017-08-17 07:00:00   4316.62   4349.99  ...          NaN           NaN
2017-08-17 08:00:00   4333.32   4377.85  ...          NaN           NaN
...                       ...       ...  ...          ...           ...
2021-12-21 06:00:00  48600.50  48704.17  ...  47438.41485  50753.723483
2021-12-21 07:00:00  48546.88  48671.56  ...  47431.04420  50739.760000
2021-12-21 08:00:00  48579.77  49000.00  ...  47425.72310  50728.685900
2021-12-21 09:00:00  48566.43  48699.00  ...  47425.69415  50718.215050
2021-12-21 10:00:00  48591.71  48625.00  ...  47424.74080  50708.524983

[37969 rows x 7 columns]


In [10]:
usdt = 100
btc = 0
lastIndex = df.first_valid_index()

for index, row in df.iterrows():
  if df['SMA200'][lastIndex] > df['SMA600'][lastIndex] and usdt > 10:
    btc = usdt / df['close'][index] 
    btc = btc - 0.0007 * btc
    usdt = 0
    print("Buy BTC at",df['close'][index],'$ the', index)

  if df['SMA200'][lastIndex] < df['SMA600'][lastIndex] and btc > 0.0001:
    usdt = btc * df['close'][index]
    usdt = usdt - 0.0007 * usdt
    btc = 0
    print("Sell BTC at",df['close'][index],'$ the', index)
  lastIndex = index

Buy BTC at 4191.53 $ the 2017-09-11 11:00:00
Sell BTC at 4259.44 $ the 2017-09-12 13:00:00
Buy BTC at 4302.02 $ the 2017-10-01 08:00:00
Sell BTC at 16351.02 $ the 2017-12-27 05:00:00
Buy BTC at 14219.61 $ the 2018-01-10 06:00:00
Sell BTC at 14195.0 $ the 2018-01-15 12:00:00
Buy BTC at 10940.0 $ the 2018-02-19 17:00:00
Sell BTC at 8704.0 $ the 2018-03-11 07:00:00
Buy BTC at 8018.98 $ the 2018-04-16 21:00:00
Sell BTC at 8665.97 $ the 2018-05-14 00:00:00
Buy BTC at 6584.36 $ the 2018-07-07 10:00:00
Sell BTC at 7087.99 $ the 2018-08-07 11:00:00
Buy BTC at 6902.74 $ the 2018-08-28 04:00:00
Sell BTC at 6355.0 $ the 2018-09-11 03:00:00
Buy BTC at 6669.36 $ the 2018-09-27 19:00:00
Sell BTC at 6309.02 $ the 2018-10-12 22:00:00
Buy BTC at 6573.58 $ the 2018-10-20 10:00:00
Sell BTC at 6491.85 $ the 2018-10-28 10:00:00
Buy BTC at 4125.4 $ the 2018-12-24 04:00:00
Sell BTC at 3588.24 $ the 2019-01-13 02:00:00
Buy BTC at 3626.58 $ the 2019-02-12 04:00:00
Sell BTC at 7660.98 $ the 2019-06-09 18:00:00


In [14]:
finalResult = usdt + btc * df['close'].iloc[-1]
print("Final result",finalResult,'USDT')

Final result 3304.4551077017404 USDT


In [15]:
print("Buy and hold result", (1000 / df['close'].iloc[0]) * df['close'].iloc[-1],'USDT')

Buy and hold result 11274.464297732797 USDT
